In [1]:
"""
Created on 24-April-2024
@author: jdmantillaq
"""

import plotly.express as px
import pandas as pd
import numpy as np
import dash
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import dash_leaflet as dl
import dash_daq as daq
from pathlib import Path
import seaborn as sns

In [76]:
index_prop = {
    'R5mm': {'var_values': np.linspace(0, 250),
             'anom_values': np.linspace(-30, 30),
             'cmap': 'viridis', 'units': 'días/año'},
    'R10mm': {'var_values': np.linspace(0, 200),
              'anom_values': np.linspace(-30, 30),
              'cmap': 'viridis', 'units':  'días/año'},
    'R20mm': {'var_values': np.linspace(0, 250),
              'anom_values': np.linspace(-30, 30),
              'cmap': 'viridis', 'units':  'días/año'},
    'R50mm': {'var_values': np.linspace(0, 250),
              'anom_values': np.linspace(-30, 30),
              'cmap': 'viridis', 'units': 'días/año'},
    'CDD': {'var_values': np.linspace(0, 60),
            'anom_values': np.linspace(-10, 10),
            'cmap': 'plasma', 'units': 'días/año'},
    'CWD': {'var_values': np.linspace(0, 60),
            'anom_values': np.linspace(-10, 10),
            'cmap': 'plasma', 'units': 'días/año'},

    'R95p': {'var_values': np.linspace(0, 1000),
             'anom_values': np.linspace(-350, 350),
             'cmap': "Spectral_r",
             'units': 'mm/año'},
    'R99p': {'var_values': np.linspace(0, 400),
             'anom_values': np.linspace(-250, 250),
             'cmap': "Spectral_r",
             'units': 'mm/año'},
    'PRCPTOT': {'var_values': np.linspace(0, 6000),
                'anom_values': np.linspace(-900, 900),
                'cmap': "Spectral_r",
                'units': 'mm/año'},

    'SDII': {'var_values': np.linspace(0, 30),
             'anom_values': np.linspace(-5, 5),
             'cmap': "Spectral_r",
             'units': 'mm/día'},

    'P75y': {'var_values': np.linspace(0, 60),
             'anom_values': np.linspace(-8, 8),
             'cmap': "Spectral_r",
             'units': 'mm/día'},
    'P90y': {'var_values': np.linspace(0, 60),
             'anom_values': np.linspace(-8, 8),
             'cmap': "Spectral_r",
             'units': 'mm/día'},
    'P95y': {'var_values': np.linspace(0, 60),
             'anom_values': np.linspace(-8, 8),
             'cmap': "Spectral_r",
             'units': 'mm/día'},

    'P75p': {'var_values': np.linspace(0, 30),
             'anom_values': np.linspace(-8, 8),
             'cmap': "Spectral_r",
             'units': 'días/año'},
    'P90p': {'var_values': np.linspace(0, 30),
             'anom_values': np.linspace(-8, 8),
             'cmap': "Spectral_r",
             'units': 'días/año'},
    'P95p': {'var_values': np.linspace(0, 30),
             'anom_values': np.linspace(-8, 8),
             'cmap': "Spectral_r",
             'units': 'días/año'},

    'TXx': {'var_values': np.linspace(18, 42),
            'anom_values': np.linspace(-6, 6),
            'cmap': 'Spectral_r',
            'units': '°C'},
    'TXn': {'var_values': np.linspace(18, 42),
            'anom_values': np.linspace(-6, 6),
            'cmap': 'Spectral_r',
            'units': '°C'},
    
    'TNn': {'var_values': np.linspace(6, 32),
            'anom_values': np.linspace(-6, 6),
            'cmap': 'Spectral_r',
            'units': '°C'},
    
    'TNx': {'var_values': np.linspace(6, 32),
            'anom_values': np.linspace(-6, 6),
            'cmap': 'Spectral_r',
            'units': '°C'},

    'TN10p': {'var_values': np.linspace(0, 15),
              'anom_values': np.linspace(-10, 10),
              'cmap': 'viridis',
              'units': '% días/año'},
    'TN90p': {'var_values': np.linspace(0, 15),
              'anom_values': np.linspace(-80, 80),
              'cmap': 'viridis',
              'units': '% días/año'},

    'TX10p': {'var_values': np.linspace(6, 15),
              'anom_values': np.linspace(-15, 15),
              'cmap': 'viridis',
              'units': '% días/año'},
    'TX90p': {'var_values': np.linspace(6, 15),
              'anom_values': np.linspace(-90, 90),
              'cmap': 'viridis',
              'units': '% días/año'},

    'DTR': {'var_values': np.linspace(6, 18),
            'anom_values': np.linspace(-1, 1),
            'cmap': 'Spectral_r',
            'units': '°C'},

    'WSDI': {'var_values': np.linspace(0, 30),
             'anom_values': np.linspace(-250, 250),
             'cmap': 'plasma',
             'units': 'días/año'},
}

In [77]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = Dash(__name__, external_stylesheets=[dbc.themes.ZEPHYR, dbc_css])
app.title = 'Cambio Climático'

load_figure_template("zephyr")
slider_style = {"font-size": "18px",
                'margin-top': '20px', 'left-right': '30px'}
text_style = {"font-size": "20px"}
colorbar_style =  {"font-size": "20px", 'color': 'black'}

keys = ["toner", "satellite", "watercolor"]  # Example
url_template = "http://{{s}}.tile.openstreetmap.org/{}/{{z}}/{{x}}/{{y}}.png"
attribution = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'



path_asses = './assets/'
img_extent = (-115+360,  -10, -30+360, 30)
image_bounds = [[-10, -115], [30, -30]]
# map_dl = dl.Map([dl.TileLayer(),
#                  dl.FullScreenControl(),
#                  dl.LayersControl(
#     [dl.BaseLayer(dl.TileLayer(url=url_template.format(key)),
#                   name=key, checked=key == "watercolor") for key in keys]),
#     dl.ImageOverlay(opacity=0.7,
#                     url="/assets/ERA5_TXx_historical_1979-2014.png",
#                     bounds=image_bounds),
#     dl.ScaleControl(position="bottomright"),
#     dl.Colorbar(colorscale=colorscale, width=20,
#                 height=250,
#                 nTicks = 5,            
#                 min=0,
#                 max=50, position="topleft",
#                 style=colorbar_style,
#                 tooltip=True,
#                 unit='°C')],
#     zoom=4.5, center=(6.248011, -75.577277),
#     style={'width': '100%',
#            'height': '75vh',
#            'margin': "auto",
#            "display": "block"})

map_content_basic = [dl.TileLayer(),
        dl.FullScreenControl(),
        dl.ScaleControl(position="bottomright"),
        ]




# Define the layout
map_dl = app.layout = html.Div([
    dl.Map(id='map', children=map_content_basic,
        zoom=4.5, center=(6.248011, -75.577277),
        style={'width': '100%',
               'height': '75vh',
               'margin': "auto",
               "display": "block"})
])


# map_dl = html.Div()

# -----------------------------------------------------------------------------
# popover Datasets
# -----------------------------------------------------------------------------

models_NEX = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CanESM5',
              'CMCC-CM2-SR5', 'CMCC-ESM2', 'EC-Earth3',
              'EC-Earth3-Veg-LR', 'GFDL-CM4',
              'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0',
              'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM', 'MIROC6',
              'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3',
              'NorESM2-LM', 'NorESM2-MM', 'TaiESM1']

models_NEX = ['BCC-CSM2-MR', 'MIROC6', 'MPI-ESM1-2-LR',  'MRI-ESM2-0',
              'NorESM2-LM']


models_HighResp = ['FGOALS-f3-L', 'MRI-AGCM3-2-H',
                   'MRI-AGCM3-2-S', 'NICAM16-7S', 'NICAM16-8S']

obs_list = ['ERA5', 'CHIRPS']

radioitems_dataset_obs = dbc.CardBody(dcc.RadioItems(obs_list,
                                                     id="radio_dataset_obs",
                                                     ),
                                      style=slider_style)


radioitems_dataset_NEX = dbc.CardBody(dcc.RadioItems(models_NEX,
                                                     id="radio_dataset_NEX",),
                                      style=slider_style)

radioitems_dataset_HighResp = dbc.CardBody(
    dcc.RadioItems(models_HighResp,
                   id="radio_dataset_HighResp",),
    style=slider_style)


content_dataset = dbc.PopoverBody(
    children=[dbc.Row([dbc.Col([html.H5(children="Observaciones")], width=4),
                       dbc.Col([html.H5(children="NEX-GDDP-CMIP6")], width=4),
                       dbc.Col([html.H5(children="HighResMIP-CMIP6")], width=4),
                       ]),
              html.Hr(),
              dbc.Row([dbc.Col([radioitems_dataset_obs], width=4),
                       dbc.Col([radioitems_dataset_NEX], width=4),
                       dbc.Col([radioitems_dataset_HighResp], width=4),
                       ])])


popovers_dataset = html.Div(
    [
        dbc.Button(
            "Dataset",
            id="button_dataset",
            n_clicks=0,
            style=slider_style
        ),
        dbc.Popover(
            [
                dbc.PopoverHeader("Seleccione un conjunto de datos"),
                dbc.PopoverBody(content_dataset),
            ],
            target="button_dataset",
            placement='bottom',
            trigger="legacy",
            autohide=True,
            style={'maxWidth': '3000px'},
            body=True,
        ),
    ]
)

# # Add the tooltip
# tool_tip_var = dbc.Tooltip("Seleccione un conjunto de datos",
#                            target="button_dataset",
#                            placement='bottom')  # Set placement to 'bottom'

# -----------------------------------------------------------------------------
# popover extreme indexes
# -----------------------------------------------------------------------------
tas_index_list = ['TXx', 'TXn', 'TNx', 'TNn', 'TX90p', 'TX10p', 'TN90p',
                  'TN10p', 'DTR', 'WSDI']

pr_index_list = ['R5mm', 'R10mm', 'R20mm', 'R50mm', 'CDD', 'CWD', 'R95p',
                 'R99p', 'PRCPTOT', 'SDII', 'P75y', 'P90y', 'P95y',
                 'P75p', 'P90p', 'P95p']

radioitems_var_tas = dbc.CardBody(dcc.RadioItems(tas_index_list,
                                                 id="radio_var_tas",
                                                 value=None,
                                                 ),
                                  style=slider_style)
radioitems_var_pr = dbc.CardBody(dcc.RadioItems(pr_index_list,
                                                id="radio_var_pr",),
                                 style=slider_style)


content_variable = dbc.PopoverBody(
    style={
        "display": "flex",
        "flexDirection": "row",
        "justifyContent": "space-between"
    },
    children=[
        dbc.Col([html.H5(children="Temperatura"),
                 html.Hr(),
                 radioitems_var_tas], width=7),
        dbc.Col([html.H5(children="Precipitación"),
                 html.Hr(),
                 radioitems_var_pr], width=7),
    ]
)


dropdown_tas_index = dbc.DropdownMenu(
    label="Temperatura",
    children=radioitems_var_tas,
    color="info",
    size="lg",
    id='dropdown_tas'
)
dropdown_pr_index = dbc.DropdownMenu(
    label="Precipitación",
    children=radioitems_var_pr,
    color="info",
    size="lg",
    id='dropdown_pr'
)

popover_body_content = dbc.Row(
    [
        dbc.Col(dropdown_tas_index),
        dbc.Col(dropdown_pr_index),
    ],
    justify="between",
)

content_variable = dbc.PopoverBody(

    children=popover_body_content
)


popovers_variable = html.Div(
    [
        dbc.Button(
            "Indices de Extremos",
            id="button_variable",
            n_clicks=0,
            style=slider_style
        ),
        dbc.Popover(
            [
                dbc.PopoverHeader("Seleccione el índice a graficar"),
                dbc.PopoverBody(content_variable),
            ],
            target="button_variable",
            placement='bottom',
            trigger="legacy",
            autohide=True,
            style={'maxWidth': '500px'},
            body=True,
        ),
    ]
)


# -----------------------------------------------------------------------------
# popover time period
# -----------------------------------------------------------------------------

hist_time_tas = ['1979-2014']
hist_time_pr = ['1981-2014']
hist_time_proj = ['2021-2040', '2041-2060', '2061-2080', '2081-2100']
scenarios = ['SSP2-4.5',  'SSP5-8.5']

radioitems_time_hist = dbc.CardBody(dcc.RadioItems(options=hist_time_proj,
                                                   id="radio_time_hist"),
                                    style=slider_style)

radioitems_scenario = dbc.CardBody(dcc.RadioItems(scenarios,
                                                  id="radio_scenario",
                                                  value=scenarios[0]),
                                   style=slider_style)

toogle_quantity = daq.ToggleSwitch(
    id='toogle_quantity',
    size=60,
    value=False,
    label=['Cambio (°C)', 'Valor (°C)'],
    style={'font-size': '32px'}
)


# content_time_period = dbc.PopoverBody(
#     # style={
#     #     "display": "flex",
#     #     "flexDirection": "row",
#     #     "justifyContent": "space-between"
#     # },
#     children=[dbc.Row([dbc.Col(html.H5(children="Cantidad"), width=6),
#                       dbc.Col(toogle_quantity, width=6)]),
#               html.Hr(),
#               dbc.Row([
#                   dbc.Col([html.H5(children="Histórico"),
#                            html.Hr(),
#                       radioitems_time_hist], width=6),
#                   dbc.Col([html.H5(children="Proyecciones"),
#                            html.Hr(),
#                       radioitems_time_proj], width=6),])
#               ]
# )


# content_conditional_obs = [
#     dbc.Row([dbc.Col(html.H5(children="Cantidad"), width=5),
#              dbc.Col(toogle_quantity, width=7)]),
#     html.Hr(),
#     dbc.Row([dbc.Col(html.H5(children="Periodo"), width=5),
#              dbc.Col(radioitems_time_hist, width=7)]),
# ]

content_conditional_obs = [
    dbc.Row([dbc.Col(html.H5(children="Cantidad"), width=5),
             dbc.Col(toogle_quantity, width=7)]),
    html.Hr(),
    dbc.Row([dbc.Col([html.H5(children="Observaciones")], width=6),
             dbc.Col([html.H5(children="Periodo")], width=6),
             ]),
    html.Hr(),
    dbc.Row([dbc.Col([radioitems_scenario], width=6),
             dbc.Col([radioitems_time_hist], width=6),
             ])]


content_conditional_models = [
    dbc.Row([dbc.Col(html.H5(children="Cantidad"), width=5),
             dbc.Col(toogle_quantity, width=7)]),
    html.Hr(),
    dbc.Row([dbc.Col([html.H5(children="Escenario")], width=6),
             dbc.Col([html.H5(children="Periodo")], width=6),
             ]),
    html.Hr(),
    dbc.Row([dbc.Col([radioitems_scenario], width=6),
             dbc.Col([radioitems_time_hist], width=6),
             ])]

content_time_period = dbc.PopoverBody(
    children=content_conditional_obs,
    id='popup_quantity_period')


popovers_time_period = html.Div(
    [
        dbc.Button(
            "Cantidad/Periodo de estudio",
            id="button_time_period",
            n_clicks=0,
            style=slider_style
        ),
        dbc.Popover(
            [dbc.PopoverHeader("Seleccione la cantidad y el periodo de estudio"),
             dbc.PopoverBody(content_time_period),
             ],
            target="button_time_period",
            placement='bottom',
            trigger="legacy",
            autohide=True,
            style={'maxWidth': '1000px'},
            body=True,
        ),
    ]
)


# -----------------------------------------------------------------------------
# Tab 1: content
# -----------------------------------------------------------------------------


tab1_content = dbc.Card(
    dbc.CardBody(
        [dbc.Row([dbc.Col([popovers_dataset, popovers_variable,
                           popovers_time_period],
                          style={"display": "flex", "flexWrap": "wrap"})]),
         dbc.Row(map_dl),
         html.H4(children="",
                 style={"padding": "10px", "text-align": "center"},
                 id='title_page'),]
    ))


# -----------------------------------------------------------------------------
# Tab 1: content
# -----------------------------------------------------------------------------


tab2_content = dbc.Card()


app.layout = html.Div(
    [html.Div(
        className="header",
        children=[
            html.Div(
                className="div-info",
                children=[
                    html.H4(children="Cambio Climático",
                            style={"padding": "10px"}),
                    # html.Hr(),
                    dbc.Tabs([
                        dbc.Tab(tab1_content, label="Índices de Extremos",
                                tab_id="tab-1")],
                             id="tabs",
                             active_tab="tab-1",
                             )
                ],
            ),
        ],
    ),
    ], style=text_style
)


# -----------------------------------------------------------------------------
# Dataset
# -----------------------------------------------------------------------------
@app.callback(
    [Output("radio_dataset_obs", "value"),
     Output("radio_dataset_NEX", "value"),
     Output("radio_dataset_HighResp", "value")],
    [Input("radio_dataset_obs", "value"),
     Input("radio_dataset_NEX", "value"),
     Input("radio_dataset_HighResp", "value")]
)
def clear_selections_dataset(obs_value, nex_value, high_value):
    ctx = dash.callback_context
    if not ctx.triggered:
        return [None, None, None]
    else:
        prop_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if prop_id == 'radio_dataset_obs':
            return [obs_value, None, None]
        elif prop_id == 'radio_dataset_NEX':
            return [None, nex_value, None]
        elif prop_id == 'radio_dataset_HighResp':
            return [None, None, high_value]
        else:
            return [None, None, None]

# -----------------------------------------------------------------------------
# Indices
# -----------------------------------------------------------------------------


@app.callback(
    [Output("radio_var_tas", "value"),
     Output("radio_var_pr", "value"),
     ],
    [Input("radio_var_tas", "value"),
     Input("radio_var_pr", "value"),
     ]
)
def clear_selections_var(value_1, value_2):
    ctx = dash.callback_context
    if not ctx.triggered:
        return [None, None]
    else:
        prop_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if prop_id == 'radio_var_tas':
            return [value_1, None]
        elif prop_id == 'radio_var_pr':
            return [None, value_2]
        else:
            return [None, None]

# -----------------------------------------------------------------------------
# Extreme Index conditional
# -----------------------------------------------------------------------------


# @app.callback(
#     Output("title_page", "children"),
#     [
#         Input("radio_dataset_obs", "value"),
#         Input("radio_dataset_NEX", "value"),
#         Input("radio_dataset_HighResp", "value"),
#         Input("radio_var_tas", "value"),
#         Input("radio_var_pr", "value"),
#         Input("radio_time_hist", "value"),
#         Input("radio_scenario", "value"),
#     ],
# )
# def change_title(obs_value, nex_value, high_value, tas_value, pr_value,
#                  time_hist_value, radio_scenario):
#     # Combine data selection logic
#     data_part = obs_value or nex_value or high_value
#     if not any([data_part]):
#         raise PreventUpdate

#     # Combine index selection logic
#     index_part = tas_value or pr_value
#     if not any([index_part]):
#         raise PreventUpdate

#     scenario = radio_scenario if radio_scenario else ""
    
#     name_file = f'{data_part}_{index_part}_{scenario}_{time_hist_value}.png'
#     file_path = Path(f".{path_asses}{name_file}")
    
#     if file_path.exists():
#         file_exists = "File exists!"
#     else:
#         file_exists = "File doesn't exists!"

#     # Construct the title with proper formatting
#     title = f"{data_part} | {index_part} | {scenario} | {time_hist_value}: {file_exists}"

#     return title


def create_possible_options(lst, disabled=False):
    color = 'grey' if disabled else 'black'
    font_size = 15 if disabled else 16

    return [{'label': i, 'style': {'color': color,
                                   'font-size': f"{font_size}px"},
             'value': i, 'disabled': disabled} for i in lst]


@app.callback(
    [Output("radio_var_tas", "options"),
     Output("radio_var_pr", "options"),
     Output("dropdown_tas", "disabled"),
     Output("dropdown_pr", "disabled")],
    [Input("radio_dataset_obs", "value"),
     Input("radio_dataset_NEX", "value"),
     Input("radio_dataset_HighResp", "value")]
)
def filter_index_available(obs_value, nex_value, high_value):
    if not any([obs_value, nex_value, high_value]):
        raise PreventUpdate

    options_tas_disabled = options_pr_disabled = False
    if obs_value is not None:
        options_tas_disabled = obs_value == 'CHIRPS'
        options_pr_disabled = obs_value == 'ERA5'

    else:
        options_tas_disabled = options_pr_disabled = False

    options_tas = create_possible_options(tas_index_list,
                                          disabled=options_tas_disabled)
    options_pr = create_possible_options(pr_index_list,
                                         disabled=options_pr_disabled)
    return options_tas, options_pr, options_tas_disabled, options_pr_disabled


@app.callback(
    Output("toogle_quantity", "label"),
    [Input("radio_dataset_obs", "value"),
     Input("radio_var_tas", "value"),
     Input("radio_var_pr", "value")]
)
def change_toogle_labels(obs_value, tas_value, pr_value):
    if not any([obs_value, tas_value, pr_value]):
        raise PreventUpdate

    word_left = 'Cambio'
    word_units_left_2 = ''
    word_right = 'Valor'
    units = '°C'

    if obs_value is not None:
        word_left = 'Tendencia'
        word_units_left_2 = '/año'
    if tas_value is not None:
        units = index_prop[tas_value]['units']
    elif pr_value is not None:
        units = index_prop[pr_value]['units']
    label = [f'{word_left} ({units}{word_units_left_2})',
             f'{word_right} ({units})']
    return label


@app.callback(
    Output("popup_quantity_period", "children"),
    [Input("radio_dataset_obs", "value"),
     Input("radio_dataset_NEX", "value"),
     Input("radio_dataset_HighResp", "value")]
)
def change_content_popup_quantity_period(obs_value, nex_value, high_value):
    if not any([obs_value, nex_value, high_value]):
        raise PreventUpdate

    if obs_value is not None:
        return content_conditional_obs
    else:
        return content_conditional_models


@app.callback(
    [Output("radio_scenario", "options"),
     Output("radio_scenario", "value"),
     Output("radio_time_hist", "options"),
     Output("radio_time_hist", "value")],

    [Input("radio_dataset_obs", "value"),
     Input("radio_dataset_NEX", "value"),
     Input("radio_dataset_HighResp", "value")]
)
def change_options_period(obs_value, nex_value, high_value):
    if obs_value == 'ERA5':
        return [], 'historical', hist_time_tas, hist_time_tas[0],
    if obs_value == 'CHIRPS':
        return [], 'historical', hist_time_pr, hist_time_pr[0],

    elif any([nex_value, high_value]):
        return scenarios, scenarios[0], hist_time_proj, hist_time_proj[0]
    else:
        raise PreventUpdate
    
#------------------------------------------------------------------------------
# Map Callbacks
#------------------------------------------------------------------------------
    
@app.callback(
    [Output("title_page", "children"),
    Output("map", "children")],
    [
        Input("radio_dataset_obs", "value"),
        Input("radio_dataset_NEX", "value"),
        Input("radio_dataset_HighResp", "value"),
        Input("radio_var_tas", "value"),
        Input("radio_var_pr", "value"),
        Input("radio_time_hist", "value"),
        Input("radio_scenario", "value"),
    ],
)
def verify_components(obs_value, nex_value, high_value, tas_value, pr_value,
                 time_hist_value, radio_scenario):
    # Combine data selection logic
    data_part = obs_value or nex_value or high_value
    if not any([data_part]):
        raise PreventUpdate

    # Combine index selection logic
    index_part = tas_value or pr_value
    if not any([index_part]):
        raise PreventUpdate

    scenario = radio_scenario if radio_scenario else ""
    
    name_file = f'{data_part}_{index_part}_{scenario}_{time_hist_value}.png'
    file_path = f"{path_asses}{name_file}"
    
    file_exists = "File exists!" if Path(file_path).exists() else "File doesn't exist!"
    
    # Construct the title with proper formatting
    title = f"{data_part} | {index_part} | {scenario} | {time_hist_value}: {file_exists}"
    
    if not Path(file_path).exists():
        return title, map_content_basic

    map_content_updated = update_map(index_part, file_path)

    return title, map_content_updated



def update_map(index, file):
    
    cmap = index_prop[index]['cmap']
    levels = index_prop[index]['var_values']
    units = index_prop[index]['units']
    
    colorscale = sns.palettes.color_palette(cmap).as_hex()
    map_content = [dl.TileLayer(),
            dl.FullScreenControl(),
            dl.ImageOverlay(id='image-overlay',
                            opacity=0.7,
                            url=file,
                            bounds=image_bounds),
            dl.ScaleControl(position="bottomright"),
            dl.Colorbar(id='colorbar',
                        colorscale=colorscale,
                        width=20,
                        height=250,
                        nTicks=5,
                        style=colorbar_style,
                        min=levels[0],
                        max=levels[-1],
                        position="bottomleft",
                        tooltip=True,
                        unit=units)]
    return map_content


app.run_server(port=2100, host='0.0.0.0')

In [59]:
file_path = Path(f".{path_asses}ERA5_TXx_historical_1979-2014.png")

if file_path.exists():
    print("File exists!")
else:
    print("File does not exist.")

File exists!


In [53]:
file_path

PosixPath('/assets/ERA5_TXx_historical_1979-2014.png')

In [4]:
# %% #########  Actualización de trayectorias #########
@app.callback(
    [Output('actual-date', 'children'),
     Output("map-1", "children")],
    [Input('interval-component', 'n_intervals'),
     Input("step-slider", "value")])
def update_components(n, step):
    """
    Esta funcion actualiza todos los componentes del dash con el intervalo especificado
    """
    #########  Se hacen globales los dataframe #########
    global GFS
    global NCEP
    global pronostico_0
    global pronostico_1
    global pronostico_2

    # %% ######### Lectura de trayectorias GFS #########
    GFS = read_csv(path_data + 'trajectories_GFS.csv',
                   parse_dates=['date']).dropna(how='any')

    # %% ######### Lectura de trayectorias GFS #########
    NCEP = read_csv(path_data + 'trajectories_NCEP.csv',
                    parse_dates=['date']).dropna(how='any')

    # %% ######### Pronostico Trayectirias GFS #########
    pronostico_0 = read_csv(
        path_data + 'trajectories_pronostico_0.csv', parse_dates=['date'])
    pronostico_1 = read_csv(
        path_data + 'trajectories_pronostico_1.csv', parse_dates=['date'])
    pronostico_2 = read_csv(
        path_data + 'trajectories_pronostico_2.csv', parse_dates=['date'])

    # %%

    ######### Actualización de fecha #########
    fecha = update_date(n)

    mapa1 = update_map(n, step)

    return fecha, mapa1


def update_date(n_intervals):
    """
    Tiempo en el que se actulizacion 

    Parameters
    ----------
    n_intervals : int
       Este argumento recibe el tiempo en mlisegundos.

    Returns
    -------
    str
        Fecha de actualización '%Y-%m-%d %H:%M:%S'
    """
    global today

    today = datetime.today()
    return f"Fecha: {datetime.today().strftime('%Y-%m-%d %H:%M:%S')}"


def update_map(n_intervals, step):
    try:
        GFS_850 = markets(GFS, 850, step)
        GFS_800 = markets(GFS, 800, step)
        GFS_750 = markets(GFS, 750, step)

        NCEP_850 = markets(NCEP, 850, step)
        NCEP_800 = markets(NCEP, 800, step)
        NCEP_750 = markets(NCEP, 750, step)

        FORE_0_850 = markets(pronostico_0, 850, step)
        FORE_0_800 = markets(pronostico_0, 800, step)
        FORE_0_750 = markets(pronostico_0, 750, step)

        FORE_1_850 = markets(pronostico_1, 850, step)
        FORE_1_800 = markets(pronostico_1, 800, step)
        FORE_1_750 = markets(pronostico_1, 750, step)

        FORE_2_850 = markets(pronostico_2, 850, step)
        FORE_2_800 = markets(pronostico_2, 800, step)
        FORE_2_750 = markets(pronostico_2, 750, step)

        map_update = dl.Map(id="map-1",
                            children=[
                                dl.LayersControl(
                                    [dl.BaseLayer(dl.TileLayer(url=url.format(key), attribution=attribution),
                                                  name=key, checked=key == "smooth") for key in keys]
                                    + [dl.Overlay(dl.LayerGroup(GFS_850), name="GFS-850", checked=True),
                                       dl.Overlay(dl.LayerGroup(
                                           NCEP_850), name="NCEP-850", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           GFS_800), name="GFS-800", checked=True),
                                       dl.Overlay(dl.LayerGroup(
                                           NCEP_800), name="NCEP-800", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           GFS_750), name="GFS-750", checked=True),
                                       dl.Overlay(dl.LayerGroup(
                                           NCEP_750), name="NCEP-750", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_0_850), name="FORE0-850", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_0_800), name="FORE0-800", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_0_750), name="FORE0-750", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_1_850), name="FORE1-850", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_1_800), name="FORE1-800", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_1_750), name="FORE1-750", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_2_850), name="FORE2-850", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_2_800), name="FORE2-800", checked=False),
                                       dl.Overlay(dl.LayerGroup(
                                           FORE_2_750), name="FORE2-750", checked=False),
                                       dl.Overlay(dl.ImageOverlay(opacity=0.5, url="./assets/GPM_0.png",
                                                                  bounds=[
                                                                      [-10, -82], [16, -35]]
                                                                  ), name="GPM 0", checked=True),
                                       dl.Overlay(dl.ImageOverlay(opacity=0.5, url="./assets/GPM_1.png",
                                                                  bounds=[
                                                                      [-10, -82], [16, -35]]
                                                                  ), name="GPM -1", checked=False),
                                       dl.Overlay(dl.ImageOverlay(opacity=0.5, url="./assets/GPM_2.png",
                                                                  bounds=[
                                                                      [-10, -82], [16, -35]]
                                                                  ), name="GPM -2", checked=False)
                                       ]
                                ),
                                dl.Colorbar(
                                    colorscale=aod_scale, width=30, height=175, min=0, max=.8, position="topleft"),
                                dl.Colorbar(
                                    colorscale=ppt_scale, width=30, height=175, min=0, max=500, position="topleft")
                            ],

                            zoom=4, center=(6.248011, -75.577277),
                            style={'width': '100%',
                                   'height': '70vh',
                                   'margin': "auto",
                                   "display": "block"})
        # map_update.children += [dl.locate_control(options={'enableHighAccuracy': True})]

        return map_update
    except:
        return map_component

{'id': 'radio_time_hist', 'property': 'value'}

In [5]:
radioitems_time_proj = dbc.CardBody(dcc.RadioItems(options=hist_time_proj,
                                                   id="radio_time_proj",),
                                    style=slider_style)

radioitems_scenario = dbc.CardBody(dcc.RadioItems(scenarios,
                                                  id="radio_scenario"),
                                   style=slider_style)

In [6]:
radioitems_time_proj.to_plotly_json

<bound method Component.to_plotly_json of CardBody(children=RadioItems(options=['2021-2040', '2041-2060', '2061-2080', '2081-2100'], id='radio_time_proj'), style={'font-size': '18px', 'margin-top': '20px', 'left-right': '30px'})>

In [7]:
radioitems_time_proj

CardBody(children=RadioItems(options=['2021-2040', '2041-2060', '2061-2080', '2081-2100'], id='radio_time_proj'), style={'font-size': '18px', 'margin-top': '20px', 'left-right': '30px'})

In [8]:
radioitems_time_proj

CardBody(children=RadioItems(options=['2021-2040', '2041-2060', '2061-2080', '2081-2100'], id='radio_time_proj'), style={'font-size': '18px', 'margin-top': '20px', 'left-right': '30px'})

In [9]:
radioitems_scenario

CardBody(children=RadioItems(options=['SSP2-4.5', 'SSP5-8.5'], id='radio_scenario'), style={'font-size': '18px', 'margin-top': '20px', 'left-right': '30px'})